[Reference](https://medium.com/trality/mean-reversion-trading-strategy-c944297ad148)

In [1]:
'''
This bot is a mean reversion strategy that buys and sells based on the relative value.
The cheaper the price the more is bought, and the more expensive the price the more is sold.
 
The fair price uses a kaufman adaptive moving average and the volatility is determined via the average true range.
'''
 
import numpy as np
 
SYMBOL = "DOCKUSDT"
 
# parameters for regime filter
REGIME_VOL_PERIOD=24
 
# parameter for signal
ATR_PERIODS = 24
ATR_MULTIPLIER = 2.5
KAMA_PERIODS = 24
 
# trading parameters
SPREAD = 0.05
MIN_TRADE_SIZE = 50
RISK_FACTOR = 1.0
 
def clamp(x: float, lo=0, hi=1):
   '''
   clamp a value to be within two bounds
   returns lo if x < lo, hi if x > hi else x
   '''
   return min(hi, max(x, lo))
 
'''
set up the strategy state object
'''
def initialize(state):
   state.regime_filter = 0
 
'''
set up a handler to process each bar update
'''
# @schedule(interval="1h", symbol=SYMBOL, window_size=200)
def handler(state, data):
   position = query_open_position_by_symbol(symbol=data.symbol, include_dust=False)
  
   # Check if the regime is likely to result in profits
   regime_atr = data.atr(REGIME_VOL_PERIOD)[-1]
   regime_stddev = data.stddev(REGIME_VOL_PERIOD)[-1]
   if regime_atr < regime_stddev :
       state.regime_filter = 1
   elif position is not None and position.unrealized_pnl < 0:
       state.regime_filter = -1
 
   plot_line("regime_filter", state.regime_filter, data.symbol)
 
   position = query_open_position_by_symbol(symbol=data.symbol, include_dust=False)
   plot_line("pos", 0.0 if position is None else float(position.position_value), symbol=data.symbol)
 
   kama_ma = data.kama(24)
   atr = data.atr(ATR_PERIODS)
  
   vol = max(0.25, atr[-1]/np.mean(atr.to_numpy()))
   plot_line("vol", vol, data.symbol)
 
   # sanity check
   if data is None or atr is None or kama_ma is None:
       return
  
   # plot our signals
   with PlotScope.root(data.symbol):
       plot_line("kama", kama_ma[-1])
       plot_line("kama_ubnd", kama_ma[-1]+ATR_MULTIPLIER * atr[-1])
       plot_line("kama_lbnd", kama_ma[-1]-ATR_MULTIPLIER * atr[-1])
 
   portfolio_value = float(query_portfolio_value())
  
   # compute the desired position based on the richness/cheapness
   scaled_signal = -(data.close[-1] - kama_ma[-1] ) / (atr[-1] * ATR_MULTIPLIER)
   projected_signal = (scaled_signal+1) /  2   # map from [-1,1] to [0, 1]
   signal = clamp(projected_signal, lo=0, hi=1)
  
   # compute target portfolio percent allocation based on signal
   long_target_alloc = (signal + SPREAD) * RISK_FACTOR / vol
   short_target_alloc = (signal - SPREAD) * RISK_FACTOR / vol
 
   long_target_alloc = clamp(long_target_alloc, 0, 1)
   short_target_alloc = clamp(short_target_alloc, 0, 1)
 
   if state.regime_filter != 1:
       long_target_alloc, short_target_alloc = 0.0, 0.0
 
   # calculate the trade sizes needed to achieve the desired portfolio position
   #risk_allocation = portfolio_value * RISK_FACTOR# / vol
   pos_value = 0.0 if position is None else float(position.position_value)
   buy_trade_size = max(0, (long_target_alloc * portfolio_value) - pos_value)
   sell_trade_size = min(0, (short_target_alloc * portfolio_value) - pos_value)
 
   # if the positions are too far from the desired position
   # then send orders to correct the difference
   if buy_trade_size > MIN_TRADE_SIZE:
       print(f"portfolio value {portfolio_value:.2f} buying  {buy_trade_size:+.2f} USDT" )
       adjust_position(symbol=data.symbol,weight=long_target_alloc)
   elif sell_trade_size < -MIN_TRADE_SIZE:
       print(f"portfolio value {portfolio_value:.2f} selling {sell_trade_size:+.2f} USDT" )
       adjust_position(symbol=data.symbol,weight=short_target_alloc)